In [7]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_bf9286180a564611b9385cec8bea991f.csv"
df=pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [2]:
df.shape

(286, 10)

In [3]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df=df.apply(le.fit_transform)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,0,1,2,5,0,1,2,0,2,0
1,0,2,2,3,0,1,1,1,5,0
2,0,2,2,3,0,1,1,0,2,0
3,0,4,0,2,0,1,1,1,3,0
4,0,2,2,0,0,1,1,1,4,0


In [4]:
df=df.rename(columns={0:"Class"})

y=df[["Class"]]

y.groupby("Class")["Class"].count()

Class
0    201
1     85
Name: Class, dtype: int64

In [60]:
y.groupby("Class")["Class"].count()/len(y)

Class
0    0.702797
1    0.297203
Name: Class, dtype: float64

In [5]:
X=df.drop(["Class"],axis=1)
X.head()

,1,2,3,4,5,6,7,8,9
0,1,2,5,0,1,2,0,2,0
1,2,2,3,0,1,1,1,5,0
2,2,2,3,0,1,1,0,2,0
3,4,0,2,0,1,1,1,3,0
4,2,2,0,0,1,1,1,4,0


In [23]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


test = SelectKBest(score_func=f_classif, k="all")

fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
5,27.964633
3,21.833828
4,16.988619
8,11.096221
2,4.597929
0,1.468332
6,0.980135
1,0.784878
7,0.391154


In [7]:
X_scores=scores[scores["scores"]>=10]
len(X_scores)

4

In [8]:
Xp=X.iloc[:,X_scores.index]
Xp.head()

,6,4,5,9
0,2,0,1,0
1,1,0,1,0
2,1,0,1,0
3,1,0,1,0
4,1,0,1,0


In [9]:
from imblearn.ensemble import BalancedBaggingClassifier
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [10]:
BBagging=BalancedBaggingClassifier(base_estimator=MultinomialNB(),n_estimators=15,sampling_strategy="not majority",random_state=0)
scores=cross_val_score(BBagging,Xp,np.asarray(y).reshape(y.shape[0],),cv=11)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.7027972027972028 +/- 0.11254179677923833


In [11]:
print("Median accuracy: ",np.median(scores))

Median accuracy:  0.7307692307692307


In [12]:
print("Maximum accuracy: ",np.max(scores))

Maximum accuracy:  0.8461538461538461


In [13]:
print("Average accuracy: ",np.mean(scores))

Average accuracy:  0.7027972027972028


In [14]:
from sklearn import svm
BBagging=BalancedBaggingClassifier(base_estimator=svm.SVC(C=4,kernel="poly",degree=2,random_state=0),n_estimators=15,
                                   sampling_strategy="not majority",random_state=0)
scores=cross_val_score(BBagging,Xp,np.asarray(y).reshape(y.shape[0],),cv=6)

In [15]:
print("Median accuracy: ",np.median(scores))

Median accuracy:  0.7790336879432624


In [16]:
print("Maximum accuracy: ",np.max(scores))

Maximum accuracy:  0.7916666666666666


In [17]:
print("Average accuracy: ",np.mean(scores))

Average accuracy:  0.7550975177304965


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
BBagging=BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),n_estimators=15,
                                   sampling_strategy="not majority",random_state=0)
scores=cross_val_score(BBagging,Xp,np.asarray(y).reshape(y.shape[0],),cv=5)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.7098003629764065 +/- 0.0834151933841713


In [19]:
print("Median accuracy: ",np.median(scores))

Median accuracy:  0.7068965517241379


In [20]:
print("Maximum accuracy: ",np.max(scores))

Maximum accuracy:  0.8421052631578947


In [21]:
print("Average accuracy: ",np.mean(scores))

Average accuracy:  0.7098003629764065


In [24]:
X_scores=scores[scores["scores"]>4]
len(X_scores)

5

In [25]:
Xp=X.iloc[:,X_scores.index]
Xp.head()

,6,4,5,9,3
0,2,0,1,0,5
1,1,0,1,0,3
2,1,0,1,0,3
3,1,0,1,0,2
4,1,0,1,0,0


In [26]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
Xp=pd.DataFrame(scaler.fit_transform(Xp))
Xp.head()

,0,1,2,3,4
0,1.290564,-0.536062,-0.379473,-0.558504,0.450378
1,-0.066426,-0.536062,-0.379473,-0.558504,-0.496740
2,-0.066426,-0.536062,-0.379473,-0.558504,-0.496740
3,-0.066426,-0.536062,-0.379473,-0.558504,-0.970300
4,-0.066426,-0.536062,-0.379473,-0.558504,-1.917418


In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(Xp,y,test_size=0.3,random_state=42)
print(X_train.shape)
print(X_test.shape)

(200, 5)
(86, 5)


In [28]:
from sklearn.naive_bayes import GaussianNB
NB=GaussianNB().fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
NB

GaussianNB()

In [29]:
from sklearn.metrics import accuracy_score
train_pred=NB.predict(X_train)
test_pred=NB.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.735
Accuracy on test data:  0.7325581395348837


In [30]:
from imblearn.ensemble import BalancedRandomForestClassifier 
BForest=BalancedRandomForestClassifier(n_estimators=150,sampling_strategy="not majority",
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
BForest

BalancedRandomForestClassifier(n_estimators=150, random_state=0,
                               sampling_strategy='not majority')

In [31]:
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.9
Accuracy on test data:  0.7325581395348837


In [32]:
train=pd.concat([X_train,y_train],axis=1)
train0=train[train["Class"]==0]
train1=train[train["Class"]==1]
train1=pd.concat([train1]*2,axis=0)
train=pd.concat([train0,train1],axis=0)
X_train=train.drop(["Class"],axis=1)
y_train=train[["Class"]]
print(X_train.shape)

(258, 5)


In [33]:
NB=GaussianNB().fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
NB

GaussianNB()

In [34]:
train_pred=NB.predict(X_train)
test_pred=NB.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6744186046511628
Accuracy on test data:  0.7209302325581395


In [35]:
BForest=BalancedRandomForestClassifier(n_estimators=150,sampling_strategy="not majority",
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
BForest

BalancedRandomForestClassifier(n_estimators=150, random_state=0,
                               sampling_strategy='not majority')

In [36]:
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8604651162790697
Accuracy on test data:  0.7325581395348837


In [37]:
Forest=RandomForestClassifier(n_estimators=150,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
Forest

RandomForestClassifier(n_estimators=150, random_state=0)

In [38]:
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8604651162790697
Accuracy on test data:  0.7325581395348837


In [39]:
X_train,X_test,y_train,y_test=train_test_split(Xp,y,test_size=0.33,random_state=42)
print(X_train.shape)
print(X_test.shape)

(191, 5)
(95, 5)


In [40]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [41]:
y_res.groupby("Class")["Class"].count()

Class
0    134
1    134
Name: Class, dtype: int64

In [58]:
Forest=RandomForestClassifier(n_estimators=60,random_state=0).fit(X_res,np.asarray(y_res).reshape(y_res.shape[0],) )
Forest

RandomForestClassifier(n_estimators=60, random_state=0)

In [59]:
train_pred=Forest.predict(X_res)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_res,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.914179104477612
Accuracy on test data:  0.7684210526315789


In [105]:
from sklearn.neighbors import KNeighborsClassifier
KNN=KNeighborsClassifier(n_neighbors=6,weights="distance",metric="manhattan").fit(X_res,np.asarray(y_res).reshape(y_res.shape[0],) )
KNN

KNeighborsClassifier(metric='manhattan', n_neighbors=6, weights='distance')

In [106]:
train_pred=KNN.predict(X_res)
test_pred=KNN.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_res,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.914179104477612
Accuracy on test data:  0.7578947368421053


#### The most promising results:

#### A.
1. Used only 4 features: no-caps(6),tumor-size(4),inv-nodes(5),breast-squad(9).
2. BalancedBaggingClassifier with Support Vector Machine as base estimator with C=4, polynomal kernel,degree 2, sampling_strategy = "not majority", 6-fold cross validation.

median accuracy: 77.9%, maximum accuracy: 79.2%, mean accuracy: 75.5%

#### B.
1. Used 5 features: Used only 4 features: no-caps(6),tumor-size(4),inv-nodes(5),breast-squad(9), menopause(3).
2. 67% (191 instances) for training and 33% (95 instances) for testing.
3. Used SMOTE to resample the train data. Test data was left intact.
4. RandomForestClassifier with 60 estimators.

Train data accuracy: 91.42%. Test data accuracy: 76.84%

#### C.
1. Data is preprocessed,split and resampled as above.
2. KNeighborsClassifier with 6 neighbors, "distance" weights and "manhattan" metric.

Train data accuracy: 91.42%. Test data accuracy: 75.8%